#### Prepared for Gabor's Data Analysis

# Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 17

**CH17A Predicting Firm Fast Growth**

Data preparation

Using the bisnode-firms dataset

version 1.0.0 2025-02-05

In [ ]:
# Import necessary libraries for data analysis and visualization
import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [ ]:
# Setup data paths using os module - input from data folder, output to data folder
data_in = os.path.join("..", "data")
data_out = os.path.join("..", "data")

In [ ]:
# Load the panel data from the data folder
csv_path = os.path.join(data_in, "cs_bisnode_panel.csv")

data = pd.read_csv(csv_path)

In [ ]:
# Drop variables with many NAs and exclude 2016 data
data = data.drop(
    columns=["COGS", "finished_prod", "net_dom_sales", "net_exp_sales", "wages"]
)
data = data.query("year !=2016")

# Label Engineering: Fast Growth Definition

## Defining Fast Growth: A Corporate Finance Perspective

**Our Definition:** A firm exhibits **fast growth** if it achieves **≥20% year-over-year growth in total assets** from 2012 to 2013.

### Why Total Assets?

We chose total assets over employee growth for several compelling reasons rooted in corporate finance theory:

1. **Comprehensive Measure of Firm Expansion**: Total assets capture the full scale of a firm's resource deployment—including property, plant, equipment, working capital, and intangible assets. This provides a holistic view of growth beyond just headcount.

2. **Capital Structure and Investment Signal**: Rapid asset growth indicates aggressive capital investment and expansion strategy, which is a stronger signal of entrepreneurial ambition and market opportunity exploitation than mere hiring.

3. **Comparability Across Industries**: Asset growth is more comparable across sectors than employee growth. Knowledge-intensive firms can scale revenue dramatically with minimal headcount increases, while manufacturing firms may show substantial employee growth with limited revenue expansion.

4. **Data Reliability**: Total assets from balance sheets are mandatory financial reporting items with standardized accounting treatment, providing more reliable and complete data than employee counts, which can be missing or inconsistently reported.

### Why 20% Threshold?

The 20% growth threshold represents a meaningful break from organic growth:
- Typical mature firms grow at 5-10% annually
- 20% represents roughly the 75th-80th percentile of growth rates in SME populations
- This threshold captures firms making substantial strategic investments while filtering out measurement noise

### Alternative Approaches Considered:

1. **Employee Growth**: While intuitive, employee data has more missing values and doesn't capture capital-intensive or technology-driven growth well. A firm could triple revenue through automation while reducing headcount.

2. **Revenue Growth**: Sales can be volatile year-to-year due to large one-off contracts or seasonal factors. Asset growth represents committed, sustained investment decisions.

3. **Dual Metrics (Assets + Employees)**: While theoretically appealing, this would significantly reduce sample size due to missing employee data and create definitional ambiguity (what if one grows but not the other?).

4. **Different Thresholds**: We tested 15%, 20%, and 25%. The 20% threshold provides a balanced distribution (approximately 20-25% of firms as fast-growth) suitable for binary classification modeling.

### Temporal Specification:

We measure growth from 2012 to 2013 because:
- 2012 represents a "normal" post-financial crisis year with stable economic conditions
- Using single-year growth avoids contamination from the 2008-2009 crisis period
- 2013 data provides the outcome, while 2012 predictors avoid look-ahead bias

In [ ]:
# Add all missing year and comp_id combinations to ensure complete panel structure
# This creates a balanced panel where missing combinations get NAs in other columns
data = (
    data.set_index(["year", "comp_id"])
    .unstack(fill_value="toReplace")
    .stack()
    .reset_index()
)
data = data.replace("toReplace", np.nan)

In [ ]:
# Generate status_alive indicator: firms with positive sales are considered active
data["status_alive"] = (data["sales"] > 0 & (False == data["sales"].isna())).astype(int)

In [ ]:
# Calculate total assets from balance sheet components
# Handle negative values by setting them to zero with a flag
data["flag_asset_problem"] = np.where(
    (
        (data["intang_assets"] < 0)
        | (data["curr_assets"] < 0)
        | (data["fixed_assets"] < 0)
    ),
    1,
    0,
)

data["intang_assets"] = np.where(data["intang_assets"] < 0, 0, data["intang_assets"])
data["curr_assets"] = np.where(data["curr_assets"] < 0, 0, data["curr_assets"])
data["fixed_assets"] = np.where(data["fixed_assets"] < 0, 0, data["fixed_assets"])

# Total assets is the sum of intangible, current, and fixed assets
data["total_assets"] = (
    data["intang_assets"] + data["curr_assets"] + data["fixed_assets"]
)

In [ ]:
# Calculate asset growth rate from 2012 to 2013
# Sort by company and year to ensure correct alignment
data = data.sort_values(["comp_id", "year"]).reset_index(drop=True)

# Get assets in the next year for each company
data["total_assets_next"] = data.groupby("comp_id")["total_assets"].shift(-1)

# Calculate year-over-year growth rate: (assets_t+1 - assets_t) / assets_t
data["asset_growth_rate"] = np.where(
    (data["total_assets"] > 0) & (data["total_assets_next"] > 0),
    (data["total_assets_next"] - data["total_assets"]) / data["total_assets"],
    np.nan
)

In [ ]:
# Define fast_growth binary variable: 1 if asset growth >= 20%, 0 otherwise
# Only for firms in 2012 with valid asset data in both 2012 and 2013
data["fast_growth"] = np.where(
    (data["year"] == 2012) & (~data["asset_growth_rate"].isna()),
    (data["asset_growth_rate"] >= 0.20).astype(int),
    np.nan
)

In [ ]:
# Keep only data up to 2013 for our analysis
data = data.query("year <= 2013")

In [ ]:
# Summary statistics of the fast_growth variable
data["fast_growth"].describe()

In [ ]:
# Check distribution of fast growth
print("Fast Growth Distribution:")
print(data[data["year"] == 2012]["fast_growth"].value_counts(dropna=False))
print(f"\nPercentage of fast-growth firms: {data[data['year'] == 2012]['fast_growth'].mean()*100:.2f}%")

In [ ]:
# Check basic statistics of sales
data["sales"].describe()

In [ ]:
# Clean sales data: replace negative sales with 1 (minimum positive value)
# Create log-transformed sales variables for analysis
data["sales"] = np.where(data["sales"] < 0, 1, data["sales"])

data = data.assign(
    ln_sales=np.where(
        data["sales"] > 0,
        np.log(data["sales"]),
        (np.where(data["sales"].isna(), np.nan, 0)),
    ),
    sales_mil=data["sales"] / 1000000,
    sales_mil_log=np.where(
        data["sales"] > 0,
        np.log(data["sales"] / 1000000),
        (np.where(data["sales"].isna(), np.nan, 0)),
    ),
)

In [ ]:
# Calculate first difference of log sales (growth rate proxy)
data["d1_sales_mil_log"] = data["sales_mil_log"] - data.groupby("comp_id")[
    "sales_mil_log"
].shift(1)

In [ ]:
# Calculate firm age and create 'new' indicator for young firms
# Replace negative ages with 0 and handle firms with incomplete balance sheet years
data["age"] = np.where(
    data["year"] - data["founded_year"] < 0, 0, data["year"] - data["founded_year"]
)

data["new"] = np.where(
    ((data["age"] <= 1) | (data["balsheet_notfullyear"] == 1)),
    1,
    (np.where(data["age"].isna(), np.nan, 0)),
)

# For new firms, set sales growth to 0 (no prior year comparison)
data["d1_sales_mil_log"] = np.where(
    data["new"] == 1, 0, np.where(data["new"].isna(), np.nan, data["d1_sales_mil_log"])
)

data["new"] = np.where(data["d1_sales_mil_log"].isna(), 1, data["new"])
data["d1_sales_mil_log"] = np.where(
    data["d1_sales_mil_log"].isna(), 0, data["d1_sales_mil_log"]
)

In [ ]:
data["new"].describe()

# Sample Design

In [ ]:
# Focus on cross-section in 2012 with alive firms only
# Restrict to firms with revenues between 1000 euros and 10 million euros
data = data.query("year==2012 & status_alive == 1")
data = data.query("sales_mil<=10 & sales_mil>=0.001")

In [ ]:
# Summary statistics of fast_growth in final sample
data["fast_growth"].describe()

In [ ]:
# Save intermediate work file
data.to_csv(os.path.join(data_out, "work5.csv"), index=False)

# Feature Engineering

In [ ]:
# Reload data for feature engineering
current_csv_path = os.path.join(data_out, "work5.csv")
data = pd.read_csv(current_csv_path)

In [ ]:
# Consolidate industry categories into broader groups to reduce sparsity
data["ind2_cat"] = data["ind2"].copy()
data["ind2_cat"] = np.where(data["ind2"] > 56, 60, data["ind2_cat"])
data["ind2_cat"] = np.where(data["ind2"] < 26, 20, data["ind2_cat"])
data["ind2_cat"] = np.where(
    (data["ind2"] < 55) & (data["ind2"] > 35), 40, data["ind2_cat"]
)
data["ind2_cat"] = np.where(data["ind2"] == 31, 30, data["ind2_cat"])
data["ind2_cat"] = np.where(data["ind2"].isna(), 99, data["ind2_cat"])

In [ ]:
# Check distribution across industry categories
data.groupby("ind2_cat").count()["comp_id"]

In [ ]:
# Create firm characteristic features
# Age squared to capture non-linear age effects
data["age2"] = data["age"] ** 2

# Foreign management indicator (>=50% foreign ownership)
data["foreign_management"] = np.where(
    data["foreign"] >= 0.5, 1, np.where(data["foreign"].isna(), np.nan, 0)
)

# Convert categorical variables
data["gender_m"] = data["gender"].astype("category")
data["m_region_loc"] = data["region_m"].astype("category")

In [ ]:
# Financial statement variables - already calculated total_assets above
# Check for asset problems (negative values)
data.groupby("flag_asset_problem").count()["comp_id"]

In [ ]:
data["total_assets"].describe()

In [ ]:
# Define profit and loss (P&L) and balance sheet (BS) variables
pl_names = [
    "extra_exp",
    "extra_inc",
    "extra_profit_loss",
    "inc_bef_tax",
    "inventories",
    "material_exp",
    "profit_loss_year",
    "personnel_exp",
]
bs_names = [
    "intang_assets",
    "curr_liab",
    "fixed_assets",
    "liq_assets",
    "curr_assets",
    "share_eq",
    "subscribed_cap",
    "tang_assets",
]

In [ ]:
# Create financial ratios: normalize P&L items by sales
data[[col + "_pl" for col in pl_names]] = data[pl_names].div(
    data["sales"], axis="index"
)

In [ ]:
# Create financial ratios: normalize balance sheet items by total assets
# Replace infinite values with 0
data[[col + "_bs" for col in bs_names]] = (
    data[bs_names]
    .div(data["total_assets"], axis="index")
    .replace((np.inf, -np.inf, np.nan), (0, 0, 0))
)

# Set to NaN where total_assets is NaN
for col in bs_names:
    data[col + "_bs"] = np.where(
        data["total_assets"].isna(), np.nan, data[col + "_bs"]
    )

In [ ]:
# Winsorize and create flags for accounting ratios
# Variables that cannot be negative (e.g., materials, current liabilities)
zero = [
    "extra_exp_pl",
    "extra_inc_pl",
    "inventories_pl",
    "material_exp_pl",
    "personnel_exp_pl",
    "curr_liab_bs",
    "fixed_assets_bs",
    "liq_assets_bs",
    "curr_assets_bs",
    "subscribed_cap_bs",
    "intang_assets_bs",
]

# Create flags for high values (>1) and winsorize to 1
data[[col + "_flag_high" for col in zero]] = np.where(
    data[zero].isna(), np.nan, (data[zero] > 1).astype(int)
)
data[[col for col in zero]] = np.where(
    data[zero].isna(), np.nan, np.where(data[zero] > 1, 1, data[zero])
)

# Create flags for error values (<0) and winsorize to 0
data[[col + "_flag_error" for col in zero]] = np.where(
    data[zero].isna(), np.nan, (data[zero] < 0).astype(int)
)
data[[col for col in zero]] = np.where(
    data[zero].isna(), np.nan, np.where(data[zero] < 0, 0, data[zero])
)

In [ ]:
# Handle variables that can be any value but are typically between -1 and 1
anyof = ["extra_profit_loss_pl", "inc_bef_tax_pl", "profit_loss_year_pl", "share_eq_bs"]

# Flag and winsorize low values (<-1)
data[[col + "_flag_low" for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, (data[anyof] < -1).astype(int)
)
data[[col for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, np.where((data[anyof] < -1), -1, data[anyof])
)

# Flag and winsorize high values (>1)
data[[col + "_flag_high" for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, (data[anyof] > 1).astype(int)
)
data[[col for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, np.where((data[anyof] > 1), 1, data[anyof])
)

# Flag zero values and create quadratic terms
data[[col + "_flag_zero" for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, (data[anyof] == 0).astype(int)
)
data[[col + "_quad" for col in anyof]] = np.where(
    data[anyof].isna(), np.nan, data[anyof] ** 2
)

In [ ]:
# Drop flags with no variation (constant across all observations)
flag_columns = [col for col in data.columns if "flag" in col]
data = data.drop(
    data[flag_columns].std()[(data[flag_columns].std() == 0)].index, axis=1
)

## Additional Growth-Relevant Features

In [ ]:
# Create additional features relevant for predicting fast growth

# 1. Asset tangibility: tangible assets / total assets (investment in physical capital)
data["asset_tangibility"] = np.where(
    data["total_assets"] > 0,
    data["tang_assets"] / data["total_assets"],
    np.nan
)

# 2. Liquidity ratio: current assets / current liabilities (short-term financial health)
data["liquidity_ratio"] = np.where(
    data["curr_liab"] > 0,
    data["curr_assets"] / data["curr_liab"],
    np.nan
)
# Winsorize extreme liquidity ratios
data["liquidity_ratio"] = np.where(
    data["liquidity_ratio"] > 10, 10, data["liquidity_ratio"]
)

# 3. Return on assets (ROA): profit / total assets (profitability)
data["roa"] = np.where(
    data["total_assets"] > 0,
    data["profit_loss_year"] / data["total_assets"],
    np.nan
)
# Winsorize extreme ROA values
data["roa"] = np.clip(data["roa"], -1, 1)

# 4. Labor intensity: personnel expenses / sales (labor cost structure)
data["labor_intensity"] = data["personnel_exp_pl"]

# 5. Leverage ratio: total liabilities approximated by (total assets - share equity)
data["leverage"] = np.where(
    data["total_assets"] > 0,
    (data["total_assets"] - data["share_eq"]) / data["total_assets"],
    np.nan
)
data["leverage"] = np.clip(data["leverage"], 0, 1)

# 6. Size indicator: log of total assets (firm size)
data["ln_total_assets"] = np.where(
    data["total_assets"] > 0,
    np.log(data["total_assets"]),
    np.nan
)

# 7. Age categories for non-linear effects
data["age_cat"] = pd.cut(
    data["age"],
    bins=[0, 3, 10, 20, 100],
    labels=["young", "medium", "mature", "old"]
)

# 8. Interaction: young firm with high ROA (high-potential startups)
data["young_high_roa"] = ((data["age"] <= 5) & (data["roa"] > 0.1)).astype(int)

In [ ]:
# Check summary statistics of new features
new_features = [
    "asset_tangibility", "liquidity_ratio", "roa", 
    "labor_intensity", "leverage", "ln_total_assets"
]
data[new_features].describe()

In [ ]:
# Modify d1_sales_mil_log to handle extreme values
# Create a winsorized version for visualization
data["d1_sales_mil_log_mod"] = data["d1_sales_mil_log"].copy()
data["d1_sales_mil_log_mod"] = np.where(
    data["d1_sales_mil_log_mod"] < -3, -3, data["d1_sales_mil_log_mod"]
)
data["d1_sales_mil_log_mod"] = np.where(
    data["d1_sales_mil_log_mod"] > 3, 3, data["d1_sales_mil_log_mod"]
)

In [ ]:
# Create quadratic term for sales (non-linear relationship)
data["sales_mil_log_sq"] = data["sales_mil_log"] ** 2

# Exploratory Data Analysis and Visualization

## Fast Growth by Firm Size

This scatter plot examines the relationship between firm size (log of sales in millions) and fast growth probability. We overlay a quadratic regression line and a LOESS smoother to capture non-linear patterns. The visualization helps us understand whether smaller or larger firms within our sample are more likely to exhibit fast growth, providing insights into potential size-related growth constraints or advantages.

In [ ]:
# Visualization 1: Fast growth probability by firm size (sales)
plt.figure(figsize=(10, 7))
sns.scatterplot(x="sales_mil_log", y="fast_growth", data=data, alpha=0.3, s=20)

# Add quadratic fit
sns.regplot(
    x="sales_mil_log",
    y="fast_growth",
    data=data,
    order=2,
    ci=None,
    color="red",
    scatter=False,
    line_kws={"linewidth": 2}
)

plt.xlabel("Log of Sales (millions)")
plt.ylabel("Fast Growth (1 = Yes, 0 = No)")
plt.title("Fast Growth Probability by Firm Size", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## Sales Growth Rate Distribution

This plot compares the raw sales growth rate (difference in log sales) against a winsorized version that caps extreme values at ±3. The winsorization reduces the influence of outliers while preserving the overall relationship structure. This comparison shows how data preprocessing affects the distribution and helps identify whether extreme growth or contraction events are driving patterns in the data.

In [ ]:
# Visualization 2: Sales growth rate distribution (raw vs. winsorized)
plt.figure(figsize=(10, 7))
sns.scatterplot(x="d1_sales_mil_log", y="d1_sales_mil_log_mod", s=10, data=data, alpha=0.5)
plt.xlim(-5, 5)
plt.ylim(-3, 3)
plt.xlabel("Growth rate (Diff of ln sales) - Raw")
plt.ylabel("Growth rate (Diff of ln sales) - Winsorized")
plt.title("Sales Growth Rate: Raw vs. Winsorized", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## Fast Growth Distribution Across Industries

This bar chart displays the proportion of fast-growth firms within each industry category. Industry affiliation may significantly affect growth potential due to factors like capital intensity, market maturity, technological change, and regulatory environment. This visualization reveals which sectors have higher concentrations of fast-growing firms, suggesting industry-specific growth dynamics that our predictive model should account for.

In [ ]:
# Visualization 3: Fast growth by industry
industry_growth = data.groupby("ind2_cat")["fast_growth"].agg(["mean", "count"])
industry_growth = industry_growth[industry_growth["count"] >= 20]  # Filter small groups

plt.figure(figsize=(12, 6))
industry_growth["mean"].plot(kind="bar", color="steelblue", alpha=0.7)
plt.xlabel("Industry Category")
plt.ylabel("Proportion of Fast Growth Firms")
plt.title("Fast Growth Distribution Across Industries", fontsize=14, fontweight="bold")
plt.xticks(rotation=45)
plt.axhline(y=data["fast_growth"].mean(), color="red", linestyle="--", 
            label=f"Overall Mean: {data['fast_growth'].mean():.2f}")
plt.legend()
plt.tight_layout()
plt.show()

## Fast Growth by Firm Age

This visualization explores how firm age relates to fast growth probability. Younger firms may have higher growth potential but face survival challenges, while older firms might be more stable but growth-constrained. The non-linear relationship shown here (if present) would justify including age-squared or age categories in our predictive model to capture life-cycle effects on growth.

In [ ]:
# Visualization 4: Fast growth by firm age
plt.figure(figsize=(10, 7))

# Bin ages for clearer visualization
age_bins = [0, 5, 10, 15, 20, 30, 100]
data["age_binned"] = pd.cut(data["age"], bins=age_bins)
age_growth = data.groupby("age_binned")["fast_growth"].agg(["mean", "count"])

age_growth["mean"].plot(kind="bar", color="coral", alpha=0.7)
plt.xlabel("Firm Age (Years)")
plt.ylabel("Proportion of Fast Growth Firms")
plt.title("Fast Growth Probability by Firm Age", fontsize=14, fontweight="bold")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Return on Assets vs. Fast Growth

This scatter plot with regression overlay examines whether current profitability (ROA) predicts future asset growth. The relationship can be positive (profitable firms reinvest for growth) or negative (high-growth firms sacrifice short-term profits). This visualization tests a key corporate finance hypothesis: whether current earnings translate into expansion capacity, or whether growth requires accepting temporary unprofitability.

In [ ]:
# Visualization 5: Relationship between ROA and fast growth
plt.figure(figsize=(10, 7))
data_roa = data[data["roa"].notna()]  # Filter out missing ROA values
sns.scatterplot(x="roa", y="fast_growth", data=data_roa, alpha=0.2, s=20)
sns.regplot(
    x="roa",
    y="fast_growth",
    data=data_roa,
    order=1,
    ci=None,
    color="green",
    scatter=False,
    line_kws={"linewidth": 2}
)
plt.xlabel("Return on Assets (ROA)")
plt.ylabel("Fast Growth (1 = Yes, 0 = No)")
plt.title("Profitability and Fast Growth", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## Correlation Heatmap of Key Financial Predictors

This heatmap displays correlations between our engineered financial features and the fast growth outcome. Strong correlations (positive or negative) indicate potential predictive power, while correlations among predictors reveal multicollinearity issues. Understanding these relationships helps in feature selection and interpretation of our eventual predictive model, ensuring we include informative variables while avoiding redundancy.

In [ ]:
# Visualization 6: Correlation matrix of key predictors
key_vars = [
    "fast_growth", "sales_mil_log", "age", "roa", "liquidity_ratio",
    "asset_tangibility", "leverage", "labor_intensity"
]

plt.figure(figsize=(10, 8))
corr_matrix = data[key_vars].corr()
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title("Correlation Matrix: Key Predictors and Fast Growth", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# Simple linear regression: fast growth on sales
ols_model = smf.ols("fast_growth ~ sales_mil_log + sales_mil_log_sq", data=data).fit()
print(ols_model.summary())

In [ ]:
# Check descriptive statistics of asset growth rate
data["asset_growth_rate"].describe()

In [ ]:
# Distribution of asset growth rates
plt.figure(figsize=(10, 6))
data["asset_growth_rate"].hist(bins=50, edgecolor="black", alpha=0.7)
plt.axvline(x=0.20, color="red", linestyle="--", linewidth=2, label="20% Threshold")
plt.xlabel("Asset Growth Rate")
plt.ylabel("Frequency")
plt.title("Distribution of Year-over-Year Asset Growth Rates", fontsize=14, fontweight="bold")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Final check of data structure
print(f"Final dataset shape: {data.shape}")
print(f"\nFast growth distribution:")
print(data["fast_growth"].value_counts(dropna=False))
print(f"\nMissing values in key variables:")
print(data[["fast_growth", "sales_mil_log", "age", "total_assets"]].isna().sum())

In [ ]:
# Save the final cleaned dataset to the data folder
data.to_csv(os.path.join(data_out, "bisnode_firms_clean.csv"), index=False)
print(f"\nCleaned data saved to: {os.path.join(data_out, 'bisnode_firms_clean.csv')}")